In [ ]:
#imports
import nltk
import pandas as pd
import re
import itertools, spacy, stanza
from collections import Counter


In [ ]:
#recursos externos - modelos do spacy e stanza, recursos do nltk
nlp_spacy = spacy.load("pt_core_news_lg")
stanza.download('pt')
nlp_stanza = stanza.Pipeline('pt')
stopwords = nltk.corpus.stopwords.words('portuguese')
stemmer = nltk.stem.RSLPStemmer()


In [ ]:
#load dos meus dados
Dados = pd.read_csv(
    'Corpus  V0- Jornalismo policial e discurso de ódio - posts_estudo_inicial.csv')



In [ ]:
#recorte dos dados para pegar só os anotados
dados1 = Dados[(Dados['Odio_Bol'] == "1") | (Dados['Odio_Bol'] == "0") | (Dados['Odio_Bol'] == '?') | (Dados['Odio_Bol'] == "*")]


In [ ]:
#curiosidade: quantos spam eu anotei?
Dados['SPAM?'].value_counts()

In [ ]:
#exploratória: como estão os dados?
print(dados1.head())

In [ ]:
#contagem bruta da anotação pra Discurso de ódio
dados1.Odio_Bol.value_counts()

In [ ]:
#contagem normalizada da anotação pra Discurso de ódio
dados1.Odio_Bol.value_counts(normalize=True)

In [ ]:
#contagem bruta da anotação pra Discurso de ódio em respostas
replies = dados1[(dados1['Reply?'] == 1)]
replies.Odio_Bol.value_counts()

In [ ]:
#contagem normalizada da anotação pra Discurso de ódio em respostas
replies.Odio_Bol.value_counts(normalize=True)

In [ ]:
#contagem bruta da anotação pra Discurso de ódio em posts originais
originais =dados1[(dados1['Reply?'] == 0)]
originais.Odio_Bol.value_counts()

In [ ]:
#contagem normalizada da anotação pra Discurso de ódio em posts originais
originais.Odio_Bol.value_counts(normalize=True)

In [ ]:
#extraindo o texto dos tweets de discurso de ódio
d_odio = dados1[(dados1["Odio_Bol"] == '1') | (dados1['Odio_Bol'] == '?') | (dados1['Odio_Bol'] == "*")]
tweets_odio = d_odio.tweet

In [ ]:
#processamento sem stemming
def processa_tweet(tweet):
    '''recebe um tweet, elimina users, stopwords'''
    lixo = r"(,|\.|!|;|:|!|/|\\|\?|\]|\}|\[|\{|\-|\_|\=|\+|\)|\(|\#|\$|\%|\&|\*|\'|\))"
    tweet_pont = re.sub(lixo,' ',tweet)
    palavras = tweet_pont.split()
    copia_limpa = []
    for palavra in palavras:
        if palavra[0] == '@' or palavra in stopwords:
            pass
        else:
            copia_limpa.append(palavra.lower())
            
    return copia_limpa

alpha = "@alertario24hrs Claro, ele é super conivente com.essa zona!"
processa_tweet(alpha)



In [ ]:
def processa_tweet_nltk(tweet,stemmer):
    '''recebe um tweet, elimina users, stopwords'''
    lixo = r"(,|\.|!|;|:|!|/|\\|\?|\]|\}|\[|\{|\-|\_|\=|\+|\)|\(|\#|\$|\%|\&|\*|\'|\))"
    tweet_pont = re.sub(lixo,' ',tweet)
    palavras = tweet_pont.split()
    copia_limpa = []
    for palavra in palavras:
        if palavra[0] == '@' or palavra in stopwords:
            pass
        else:
            #com stemmer do nlkt
            copia_limpa.append(stemmer.stem(palavra.lower()))
    return copia_limpa

alpha = "@alertario24hrs Claro, ele é super conivente com.essa zona!"
processa_tweet(alpha)


In [ ]:
def processa_tweet_spacy(tweet,nlp_spacy):
    '''recebe um tweet, elimina users, stopwords'''
    lixo = r"(,|\.|!|;|:|!|/|\\|\?|\]|\}|\[|\{|\-|\_|\=|\+|\)|\(|\#|\$|\%|\&|\*|\'|\))"
    tweet_pont = re.sub(lixo,' ',tweet)
    tweet_pont = tweet_pont.strip('  ').lower()
    copia_limpa = []
    for token in nlp_spacy(tweet_pont):
        if token.text.startswith('@') or token.text in stopwords:
            pass
        else:
            copia_limpa.append(token.lemma_)
    copia_limpa = [x for x in copia_limpa if x not in [' ','   ','  ']]
    return copia_limpa

alpha = "@alertario24hrs Claro, ele é super conivente com.essa zona!"
processa_tweet_spacy(alpha,nlp_spacy)

In [ ]:
def processa_tweet_stanza(tweet,nlp_stanza):
    '''recebe um tweet, elimina users, stopwords'''
    lixo = r"(,|\.|!|;|:|!|/|\\|\?|\]|\}|\[|\{|\-|\_|\=|\+|\)|\(|\#|\$|\%|\&|\*|\'|\))"
    tweet_pont = re.sub(lixo,' ',tweet)
    tweet_pont = tweet_pont.strip(' ')
    #tweet_limpo = ' '.join(tweet_pont)
    copia_limpa = []
    for sent in nlp_stanza(tweet_pont).sentences:
        for word in sent.words:
            if word.text.startswith('@') or word.text in stopwords:
                pass
            else:
                copia_limpa.append(word.lemma)
    #copia_limpa = [x for x in copia_limpa if x not in [' ','   ','  ']]
    return copia_limpa

alpha = "@alertario24hrs Claro, ele é super conivente com.essa zona!"
processa_tweet_stanza(alpha,nlp_stanza)

In [ ]:
#testa stanza
text = ""
pos = ""
lemma = []
for sent in nlp_stanza(alpha).sentences:
    for word in sent.words:
        text += word.text + "\t"
        pos += word.upos + "\t"
        lemma.append(word.lemma)

#print(text)
#print(pos)
print(lemma)

In [ ]:
#testa spacy
text = ""
pos = ""
lemma = ""
for token in nlp_spacy(alpha):
    text += token.text + "\t"
    pos += token.pos_ + "\t"
    lemma += token.lemma_ + "\t"
    
print(lemma)

In [ ]:
#Termos mais frequentes sem stemmer
odio_limpo = [processa_tweet(x) for x in tweets_odio]
odio_plano = list(itertools.chain(*odio_limpo))

counts = Counter(odio_plano)
print(counts)


In [ ]:
#Termos mais frequentes com stemmer NLTK
odio_limpo = [processa_tweet_nltk(x,stemmer) for x in tweets_odio]
odio_plano = list(itertools.chain(*odio_limpo))

counts = Counter(odio_plano)
print(counts)
#não pega emojis - cria palavras meio incompreensíveis.

In [117]:
#Termos mais frequentes com stemmer Spacy
odio_limpo = [processa_tweet_spacy(x,nlp_spacy) for x in tweets_odio]
odio_plano = list(itertools.chain(*odio_limpo))

counts = Counter(odio_plano)
print(counts)
#melhor resultado - considera emojis palavras,
#apesar de criar um singular para 'parabéns' que eu sei lá de onde tirou


Counter({'bandido': 41, 'parabém': 25, 'pra': 23, 'polícia': 19, 'ir': 17, 'ser': 16, '👏': 16, 'policiar': 13, 'operação': 13, 'trabalhar': 13, 'matar': 12, 'morrer': 11, 'defender': 11, 'morte': 11, 'armar': 10, 'bem': 10, 'rj': 10, '"': 10, 'q': 9, 'fazer': 9, 'jacarezinho': 9, 'todo': 9, '🤣': 9, 't': 8, 'poder': 8, 'traficante': 8, '24': 8, 'https': 7, 'co': 7, 'morto': 7, 'casar': 7, 'policial': 7, 'querer': 6, 'levar': 6, 'dar': 6, 'direito': 6, 'família': 6, 'mal': 6, 'pessoa': 6, 'dever': 6, 'ladrão': 6, 'entrar': 6, 'dessar': 6, 'filho': 6, '️': 6, 'lixar': 5, 'população': 5, 'ainda': 5, 'cadeia': 5, 'aí': 5, '    ': 5, 'civil': 5, 'cima': 5, 'merecer': 5, '🏽': 5, 'ter': 5, 'vagabundar': 5, 'bom': 5, 'dia': 5, 'alertanacional': 5, 'marginar': 5, 'guerreiro': 5, 'ficar': 5, '🤬': 5, 'gente': 4, 'fuzil': 4, 'criminoso': 4, 'militar': 4, 'menos': 4, 'mandar': 4, 'pro': 4, 'ver': 4, 'voltar': 4, 'roubar': 4, 'imprensar': 4, 'sociedade': 4, 'né': 4, 'comunidade': 4, 'humano': 4, 'mor

In [ ]:
#Termos mais frequentes com stemmer Stanza
odio_limpo = [processa_tweet_stanza(x,nlp_stanza) for x in tweets_odio]
odio_plano = list(itertools.chain(*odio_limpo))

counts = Counter(odio_plano)
print(counts)

### deu ruim com os emojis, mas parece ser bom

In [ ]:
[x.lemma_ for x in nlp('eu gosto de você')]

In [ ]:
a = 'a meu deus'
b = a.split()
c = ' '.join(b)